# 🎩 Alfred 晚宴 AI 教學筆記本（SmolAgents + LangGraph 版本）
歡迎來到這個實戰導向的筆記本，我們將一起建構一位智慧代理人 Alfred，協助你處理一場奢華晚宴的大小事！

這份筆記本提供 **雙版本實作範例**：
- 🤖 SmolAgents：快速上手的工具驅動 Agent 架構
- 🌐 LangGraph：可擴展流程圖式多工具 AI Agent

你可以依需求選擇任一版本進行實作，或兩個都試試！

In [ ]:
# ✅ 安裝必要套件
!pip install smolagents langchain langgraph huggingface_hub duckduckgo-search datasets langchain_community llama_index

In [ ]:
pip install rank_bm25

## 🤖 SmolAgents 版本：快速構建多工具代理人 Alfred

In [4]:
# 工具定義（Web Search / Weather / HF Stats / Guest Info RAG）
from smolagents import CodeAgent, HfApiModel, Tool
from huggingface_hub import list_models
from langchain_community.retrievers import BM25Retriever
from langchain.docstore.document import Document
from duckduckgo_search import DDGS
import datasets, random

# DuckDuckGo Search Tool
class DuckDuckGoSearchTool(Tool):
    name = "web_search"
    description = "Web search using DuckDuckGo"
    inputs = {"query": {"type": "string", "description": "Web search query"}}
    output_type = "string"
    def forward(self, query):
        with DDGS() as ddgs:
            results = ddgs.text(query, max_results=1)
            return results[0]['body'] if results else "No results found."

# Weather Tool
class WeatherInfoTool(Tool):
    name = "weather_info"
    description = "Returns dummy weather info for a location"
    inputs = {"location": {"type": "string", "description": "Location name"}}
    output_type = "string"
    def forward(self, location):
        data = random.choice([
            {"condition": "Sunny", "temp_c": 28},
            {"condition": "Rainy", "temp_c": 15},
            {"condition": "Windy", "temp_c": 20}
        ])
        return f"Weather in {location}: {data['condition']}, {data['temp_c']}°C"

# HF Hub Stats Tool
class HubStatsTool(Tool):
    name = "hub_stats"
    description = "Fetches most popular model from HuggingFace for a given author"
    inputs = {"author": {"type": "string", "description": "Author or org name"}}
    output_type = "string"
    def forward(self, author):
        try:
            models = list(list_models(author=author, sort="downloads", direction=-1, limit=1))
            if models:
                m = models[0]
                return f"The most downloaded model by {author} is {m.id} with {m.downloads:,} downloads."
            return f"No models found for {author}."
        except Exception as e:
            return str(e)

# Guest Info Tool
class GuestInfoRetrieverTool(Tool):
    name = "guest_info"
    description = "Retrieve guest information by name or relation"
    inputs = {"query": {"type": "string", "description": "The name or relation of the guest."}}
    output_type = "string"
    def __init__(self):
        dataset = datasets.load_dataset("agents-course/unit3-invitees", split="train")
        docs = [Document(
            page_content=f"Name: {g['name']}\nRelation: {g['relation']}\nDescription: {g['description']}\nEmail: {g['email']}",
            metadata={"name": g["name"]}) for g in dataset]
        self.retriever = BM25Retriever.from_documents(docs)
    def forward(self, query):
        results = self.retriever.get_relevant_documents(query)
        return "\n\n".join([r.page_content for r in results[:2]]) if results else "No guest info found."

In [ ]:
# 建立 Alfred（SmolAgents）
model = HfApiModel()
alfred_smol = CodeAgent(
    tools=[DuckDuckGoSearchTool(), WeatherInfoTool(), HubStatsTool(), GuestInfoRetrieverTool()],
    model=model,
    add_base_tools=True,
    planning_interval=3
)

# 測試 Alfred
query = "Tell me about Lady Ada Lovelace"
print("🎩 Alfred 回應：\n", alfred_smol.run(query))

## **🔨 實作：打造 Guestbook 工具**

### **📍 Step 1：載入與轉換資料**

我們會用 Hugging Face datasets 套件載入資料，並轉成 LangChain 可用的 Document 格式。

In [8]:
import datasets
from langchain.docstore.document import Document

guest_dataset = datasets.load_dataset("agents-course/unit3-invitees", split="train")

docs = [
    Document(
        page_content="\n".join([
            f"Name: {guest['name']}",
            f"Relation: {guest['relation']}",
            f"Description: {guest['description']}",
            f"Email: {guest['email']}"
        ]),
        metadata={"name": guest["name"]}
    )
    for guest in guest_dataset
]

In [9]:
docs

[Document(metadata={'name': 'Ada Lovelace'}, page_content="Name: Ada Lovelace\nRelation: best friend\nDescription: Lady Ada Lovelace is my best friend. She is an esteemed mathematician and friend. She is renowned for her pioneering work in mathematics and computing, often celebrated as the first computer programmer due to her work on Charles Babbage's Analytical Engine.\nEmail: ada.lovelace@example.com"),
 Document(metadata={'name': 'Dr. Nikola Tesla'}, page_content="Name: Dr. Nikola Tesla\nRelation: old friend from university days\nDescription: Dr. Nikola Tesla is an old friend from your university days. He's recently patented a new wireless energy transmission system and would be delighted to discuss it with you. Just remember he's passionate about pigeons, so that might make for good small talk.\nEmail: nikola.tesla@gmail.com"),
 Document(metadata={'name': 'Marie Curie'}, page_content='Name: Marie Curie\nRelation: no relation\nDescription: Marie Curie was a groundbreaking physicist 

這段程式碼的功能是：

- 載入資料集
- 將每位來賓轉成一份 Document 物件
- 每份 Document 包含格式化後的來賓資訊

### **🔍 Step 2：建立檢索工具**

我們用 BM25Retriever 來做文字搜尋，這是一種無需 embedding 的傳統 IR 方法，簡單又實用。

In [10]:
from smolagents import Tool
from langchain_community.retrievers import BM25Retriever

class GuestInfoRetrieverTool(Tool):
    name = "guest_info_retriever"
    description = "Retrieves detailed information about gala guests based on their name or relation."
    inputs = {
        "query": {
            "type": "string",
            "description": "The name or relation of the guest you want information about."
        }
    }
    output_type = "string"

    def __init__(self, docs):
        self.is_initialized = False
        self.retriever = BM25Retriever.from_documents(docs)

    def forward(self, query: str):
        results = self.retriever.get_relevant_documents(query)
        if results:
            return "\n\n".join([doc.page_content for doc in results[:3]])
        else:
            return "No matching guest information found."

In [12]:
# 初始化工具
guest_info_tool = GuestInfoRetrieverTool(docs)

### **🤖 Step 3：讓 Alfred 用起來！**

將工具整合進 Alfred 的代理人架構中，讓他能根據語音或指令即時查詢來賓資訊：

In [13]:
from smolagents import CodeAgent, HfApiModel

model = HfApiModel()
alfred = CodeAgent(tools=[guest_info_tool], model=model)

response = alfred.run("Tell me about our guest named 'Lady Ada Lovelace'.")

print("🎩 Alfred's Response:")
print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Tell me about our guest named 'Lady Ada Lovelace'.                                                              │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  lady_ada_info = guest_info_retriever(query="Lady Ada Lovelace")                                                  
  print(lady_ada_info)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

<ipython-input-10-425149ac6647>:20: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = self.retriever.get_relevant_documents(query)


Execution logs:
Name: Ada Lovelace
Relation: best friend
Description: Lady Ada Lovelace is my best friend. She is an esteemed mathematician and friend. She is renowned for 
her pioneering work in mathematics and computing, often celebrated as the first computer programmer due to her work
on Charles Babbage's Analytical Engine.
Email: ada.lovelace@example.com

Name: Marie Curie
Relation: no relation
Description: Marie Curie was a groundbreaking physicist and chemist, famous for her research on radioactivity.
Email: marie.curie@example.com

Name: Dr. Nikola Tesla
Relation: old friend from university days
Description: Dr. Nikola Tesla is an old friend from your university days. He's recently patented a new wireless 
energy transmission system and would be delighted to discuss it with you. Just remember he's passionate about 
pigeons, so that might make for good small talk.
Email: nikola.tesla@gmail.com

Out: None

[Step 1: Duration 39.03 seconds| Input tokens: 2,081 | Output tokens: 65]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
500 Server Error: Internal Server Error for url: 
https://router.huggingface.co/hf-inference/models/Qwen/Qwen2.5-Coder-32B-Instruct/v1/chat/completions (Request ID: 
Root=1-67f130f1-5c09f06a2892fb9f7ec84a06;5ae36838-d52d-4d2b-ba9b-b9bf5de61eed)

Model too busy, unable to get response in less than 60 second(s)

[Step 2: Duration 60.11 seconds| Input tokens: 4,162 | Output tokens: 130]

AgentGenerationError: Error in generating model output:
500 Server Error: Internal Server Error for url: https://router.huggingface.co/hf-inference/models/Qwen/Qwen2.5-Coder-32B-Instruct/v1/chat/completions (Request ID: Root=1-67f130f1-5c09f06a2892fb9f7ec84a06;5ae36838-d52d-4d2b-ba9b-b9bf5de61eed)

Model too busy, unable to get response in less than 60 second(s)

## Web search on different Agent Frameworks

In [28]:
from duckduckgo_search import DDGS

class DuckSearchSummary:
    def __call__(self, query):
        with DDGS() as ddgs:
            results = list(ddgs.text(query, max_results=1))
            return results[0]["body"] if results else "No result found."

# 測試
search_tool = MySimpleDuckSearch()
# 先搜尋，再加一個 LLM 摘要步驟
context = DuckSearchSummary()("Who is the current President of France?")
response = alfred.run(f"Based on the following web content, tell me who is the current president of France:\n\n{context}")
print("🎩 Alfred's Response:")
print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Based on the following web content, tell me who is the current president of France:                             │
│                                                                                                                 │
│ The current president of France is Emmanuel Macron, who was re-elected in 2022 for a second term. Learn about   │
│ the history, powers, functions and duties of the French presidency, and the electoral process and requirements. │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  current_president = "Emmanuel Macron"                                                                            
  final_answer(current_president)                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Emmanuel Macron

[Step 1: Duration 0.20 seconds| Input tokens: 2,131 | Output tokens: 61]

🎩 Alfred's Response:
Emmanuel Macron


In [ ]:
from llama_index.tools.duckduckgo import DuckDuckGoSearchToolSpec
from llama_index.core.tools import FunctionTool

tool_spec = DuckDuckGoSearchToolSpec()
search_tool = FunctionTool.from_defaults(tool_spec.duckduckgo_full_search)
print(search_tool("Who's the current President of France?").raw_output[-1]['body'])

In [21]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()
print(search_tool.invoke("Who's the current President of France?"))

Emmanuel Macron is a French banker and politician who was elected president of France in 2017. Macron was the first person in the history of the Fifth Republic to win the presidency without the backing of either the Socialists or the Gaullists, and he was France's youngest head of state since Napoleon. PARIS (AP) — French President Emmanuel Macron vowed Thursday to stay in office until the end of his term, due in 2027, and announced that he will name a new prime minister within days following ... Find out who the current president of France is, his political career, his actions, and his impact on the country. Stay informed about French news and presidential decisions. French President Emmanuel Macron came out fighting Thursday in his first comments following the resignation of ousted Prime Minister Michel Barnier, a day after a historic no-confidence vote at ... France has a semi-presidential system of government, offering almost a balanced power sharing between the president and the p

## **🎆 安排煙火要看天氣：Weather Tool 工具**

煙火施放需要精準的天氣判斷。我們將建立一個簡單的 **天氣查詢工具**（使用虛擬 API 範例），讓 Alfred 根據地點判斷是否適合放煙火。

In [25]:
#smolagent
from smolagents import Tool
import random

class WeatherInfoTool(Tool):
    name = "weather_info"
    description = "Fetches dummy weather information for a given location."
    inputs = {"location": {"type": "string", "description": "Location name"}}
    output_type = "string"

    def forward(self, location: str):
        weather_conditions = [
            {"condition": "Rainy", "temp_c": 15},
            {"condition": "Clear", "temp_c": 25},
            {"condition": "Windy", "temp_c": 20}
        ]
        data = random.choice(weather_conditions)
        return f"Weather in {location}: {data['condition']}, {data['temp_c']}°C"

weather_info_tool = WeatherInfoTool()

In [ ]:
#langchain
from langchain.tools import Tool

def get_weather_info(location: str) -> str:
    weather_conditions = [...]
    data = random.choice(weather_conditions)
    return f"Weather in {location}: {data['condition']}, {data['temp_c']}°C"

weather_info_tool = Tool(
    name="get_weather_info",
    func=get_weather_info,
    description="Fetches dummy weather information for a given location."
)

## **🤖 Impress the AI Guests：Hugging Face Hub Stats 工具**

這場晚宴雲集 AI 領域的巨頭人物。Alfred 要如何讓對話更有水準？當然要聊他們的模型啦！

我們會建立一個工具，查詢指定作者在 Hugging Face 上最受歡迎的模型

In [27]:
from smolagents import Tool
from huggingface_hub import list_models

class HubStatsTool(Tool):
    name = "hub_stats"
    description = "Fetches top downloaded model by author."
    inputs = {"author": {"type": "string", "description": "Author name"}}
    output_type = "string"

    def forward(self, author: str):
        try:
            models = list(list_models(author=author, sort="downloads", direction=-1, limit=1))
            if models:
                model = models[0]
                return f"The most downloaded model by {author} is {model.id} with {model.downloads:,} downloads."
            else:
                return f"No models found for author {author}."
        except Exception as e:
            return f"Error: {str(e)}"

hub_stats_tool = HubStatsTool()

In [ ]:
from langchain.tools import Tool

def get_hub_stats(author: str) -> str:
    models = list(list_models(author=author, sort="downloads", direction=-1, limit=1))
    if models:
        model = models[0]
        return f"The most downloaded model by {author} is {model.id} with {model.downloads:,} downloads."
    return f"No models found."

hub_stats_tool = Tool(
    name="get_hub_stats",
    func=get_hub_stats,
    description="Fetches Hugging Face model stats by author."
)

## **🤝 整合所有工具進 Alfred！**

現在我們有三大工具：Web Search、天氣、Hugging Face 統計。讓我們一起整合進 Alfred 的代理人模型中：

In [32]:
from smolagents import CodeAgent, HfApiModel
from smolagents import Tool  # ✅ 必須是 smolagents 提供的

from smolagents import Tool
from duckduckgo_search import DDGS

class DuckSearchTool(Tool):
    name = "web_search"
    description = "Search the web using DuckDuckGo"
    inputs = {
        "query": {
            "type": "string",
            "description": "Query to search on DuckDuckGo"
        }
    }
    output_type = "string"

    def forward(self, query: str):
        with DDGS() as ddgs:
            results = list(ddgs.text(query, max_results=1))
            return results[0]['body'] if results else "No result found."
search_tool = DuckSearchTool()  # ✅ 必須是類別實例！

class WeatherInfoTool(Tool):
    name = "weather_info"
    description = "Fetches dummy weather info."
    inputs = {
        "location": {
            "type": "string",
            "description": "The location to get weather info for."
        }
    }
    output_type = "string"

    def forward(self, location: str):
        import random
        options = ["Sunny", "Rainy", "Windy"]
        return f"The weather in {location} is {random.choice(options)}."

weather_info_tool = WeatherInfoTool()

from smolagents import Tool
from huggingface_hub import list_models

class HubStatsTool(Tool):
    name = "hub_stats"
    description = "Get top model for a HF author"
    inputs = {
        "author": {
            "type": "string",
            "description": "HuggingFace username"
        }
    }
    output_type = "string"

    def forward(self, author: str):
        try:
            models = list(list_models(author=author, sort="downloads", direction=-1, limit=1))
            if models:
                m = models[0]
                return f"Top model: {m.id} with {m.downloads:,} downloads."
            return "No models found."
        except Exception as e:
            return str(e)

hub_stats_tool = HubStatsTool()

assert isinstance(search_tool, Tool)
assert isinstance(weather_info_tool, Tool)
assert isinstance(hub_stats_tool, Tool)

In [36]:
model = HfApiModel()
alfred = CodeAgent(
    tools=[search_tool, weather_info_tool, hub_stats_tool],
    model=model
)
# response = alfred.run("What is Facebook and what's their most popular model?")
# print("🎩 Alfred's Response:", response)

#SmolAgents 版本 來看看這個多功能的Agent可不可以回答下面的隨機問題！？

In [37]:
query = "What's the weather like in Paris tonight? Will it be suitable for our fireworks display?"
response = alfred.run(query)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What's the weather like in Paris tonight? Will it be suitable for our fireworks display?                        │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
402 Client Error: Payment Required for url: 
https://router.huggingface.co/hf-inference/models/Qwen/Qwen2.5-Coder-32B-Instruct/v1/chat/completions (Request ID: 
Root=1-67f136a7-4227a87b1376d4e565b2686f;90bcfc03-9a42-4571-a355-faad37e6f42a)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly 
included credits.

[Step 1: Duration 0.05 seconds]

AgentGenerationError: Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/hf-inference/models/Qwen/Qwen2.5-Coder-32B-Instruct/v1/chat/completions (Request ID: Root=1-67f136a7-4227a87b1376d4e565b2686f;90bcfc03-9a42-4571-a355-faad37e6f42a)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.